In [172]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from lightgbm import early_stopping, log_evaluation

In [173]:
TRAIN_FILE = "datasets/superstore_train.csv"
TEST_FILE = "datasets/superstore_test.csv"
OUTPUT_FILE = "datasets/test_result.csv"
RANDOM_STATE = 2004
NFOLDS = 20

In [174]:
train = pd.read_csv(TRAIN_FILE, parse_dates=["Order Date", "Ship Date"], dayfirst=False)
test = pd.read_csv(TEST_FILE, parse_dates=["Order Date", "Ship Date"], dayfirst=False)

In [175]:
train

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7996,US-2015-165743,2015-11-20,2015-11-23,Second Class,MM-18055,Michelle Moray,Consumer,United States,Aurora,...,80013,West,OFF-BI-10001758,Office Supplies,Binders,Wilson Jones 14 Line Acrylic Coated Pressboard...,9.6120,6,0.70,-7.3692
7996,7997,US-2017-105998,2017-11-03,2017-11-05,First Class,CR-12580,Clay Rozendal,Home Office,United States,San Diego,...,92037,West,TEC-AC-10004469,Technology,Accessories,Microsoft Sculpt Comfort Mouse,199.7500,5,0.00,87.8900
7997,7998,US-2017-105998,2017-11-03,2017-11-05,First Class,CR-12580,Clay Rozendal,Home Office,United States,San Diego,...,92037,West,FUR-TA-10001095,Furniture,Tables,Chromcraft Round Conference Tables,1673.1840,12,0.20,20.9148
7998,7999,US-2014-148194,2014-05-04,2014-05-07,First Class,BS-11365,Bill Shonely,Corporate,United States,Seattle,...,98105,West,FUR-FU-10001852,Furniture,Furnishings,"Eldon Regeneration Recycled Desk Accessories, ...",12.1800,7,0.00,3.8976


In [176]:
test

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
0,8002,CA-2015-110870,2015-12-12,2015-12-15,First Class,KD-16270,Karen Daniels,Consumer,United States,Los Angeles,California,90032,West,TEC-AC-10002926,Technology,Accessories,Logitech Wireless Marathon Mouse M705,299.940,6,0.0
1,8003,CA-2015-110870,2015-12-12,2015-12-15,First Class,KD-16270,Karen Daniels,Consumer,United States,Los Angeles,California,90032,West,OFF-SU-10001225,Office Supplies,Supplies,Staple remover,25.760,7,0.0
2,8004,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,TEC-PH-10004434,Technology,Phones,Cisco IP Phone 7961G VoIP phone - Dark gray,271.900,2,0.0
3,8005,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,FUR-FU-10003878,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",45.840,3,0.0
4,8006,CA-2014-143210,2014-12-01,2014-12-03,First Class,AA-10645,Anna Andreadi,Consumer,United States,Lowell,Massachusetts,1852,East,FUR-FU-10002268,Furniture,Furnishings,Ultra Door Push Plate,9.820,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,9990,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.248,3,0.2
1989,9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.960,2,0.0
1990,9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.576,2,0.2
1991,9993,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.600,4,0.0


In [177]:
test_row_ids = test["Row ID"].values

In [178]:
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

In [179]:
for df in [train, test]:
    str_cols = df.select_dtypes(include='object').columns
    for c in str_cols:
        df[c] = df[c].astype(str).str.strip()

In [180]:
def add_date_features(df):
    df["order_year"] = df["Order Date"].dt.year
    df["order_month"] = df["Order Date"].dt.month
    df["order_day"] = df["Order Date"].dt.day
    df["order_weekofyear"] = df["Order Date"].dt.isocalendar().week.astype(int)
    df["order_dayofweek"] = df["Order Date"].dt.dayofweek
    df["order_dayofyear"] = df["Order Date"].dt.dayofyear
    df["ship_lead_days"] = (df["Ship Date"] - df["Order Date"]).dt.days
    return df

In [181]:
train = add_date_features(train)
test = add_date_features(test)

In [182]:
money_cols = ["Sales", "unit_price"]
train["unit_price"] = train["Sales"] / train["Quantity"].replace({0: np.nan})
test["unit_price"] = test["Sales"] / test["Quantity"].replace({0: np.nan})
for col in money_cols + ["unit_price"]:
    train[col] = np.log1p(train[col].fillna(0))
    test[col] = np.log1p(test[col].fillna(0))

In [183]:
prod_stats = train.groupby("Product ID")["Profit"].agg([
    ("prod_profit_mean", "mean"),
    ("prod_profit_median", "median"),
    ("prod_profit_std", "std"),
    ("prod_profit_count", "count")
]).reset_index()

In [184]:
state_stats = train.groupby("State")["Profit"].agg([
    ("state_profit_mean", "mean"),
    ("state_profit_median", "median"),
    ("state_profit_std", "std"),
    ("state_profit_count", "count")
]).reset_index()

In [185]:
cat_stats = train.groupby(["Category", "Sub-Category"])["Profit"].agg([
    ("cat_sub_profit_mean", "mean"),
    ("cat_sub_profit_count", "count")
]).reset_index()

In [186]:
cust_stats = train.groupby("Customer ID")["Profit"].agg([
    ("cust_profit_mean", "mean"),
    ("cust_profit_count", "count")
]).reset_index()

In [187]:
daily_profit = train.groupby(train["Order Date"].dt.date)["Profit"].agg(["mean", "sum", "count"]).reset_index()
daily_profit.columns = ["order_date_only", "daily_profit_mean", "daily_profit_sum", "daily_profit_count"]
daily_profit["order_date_only"] = pd.to_datetime(daily_profit["order_date_only"])

In [188]:
def merge_aggregates(df):
    df = df.merge(prod_stats, on="Product ID", how="left")
    df = df.merge(state_stats, on="State", how="left")
    df = df.merge(cat_stats, on=["Category", "Sub-Category"], how="left")
    df = df.merge(cust_stats, on="Customer ID", how="left")
    df["order_date_only"] = df["Order Date"].dt.normalize()
    df = df.merge(daily_profit, left_on="order_date_only", right_on="order_date_only", how="left")
    df.drop(columns=["order_date_only"], inplace=True)
    return df

In [189]:
train = merge_aggregates(train)
test = merge_aggregates(test)

In [190]:
global_mean = train["Profit"].mean()
global_std = train["Profit"].std()
fillna_map = {
    "prod_profit_mean": global_mean,
    "prod_profit_median": global_mean,
    "prod_profit_std": global_std,
    "prod_profit_count": 0,
    "state_profit_mean": global_mean,
    "state_profit_median": global_mean,
    "state_profit_std": global_std,
    "state_profit_count": 0,
    "cat_sub_profit_mean": global_mean,
    "cat_sub_profit_count": 0,
    "cust_profit_mean": global_mean,
    "cust_profit_count": 0,
    "daily_profit_mean": global_mean,
    "daily_profit_sum": 0,
    "daily_profit_count": 0
}

In [191]:
train.fillna(value=fillna_map, inplace=True)
test.fillna(value=fillna_map, inplace=True)

In [192]:
cat_features = ["Ship Mode", "Segment", "Country", "City", "State", "Region",
                "Category", "Sub-Category", "Product ID", "Customer ID"]

In [193]:
for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([train[col], test[col]]).astype(str)
    le.fit(combined)
    train[col + "_enc"] = le.transform(train[col].astype(str))
    test[col + "_enc"] = le.transform(test[col].astype(str))

In [194]:
target = "Profit"

numeric_cols = ["Sales", "Quantity", "Discount", "unit_price",
                "order_year", "order_month", "order_day", "order_weekofyear",
                "order_dayofweek", "order_dayofyear", "ship_lead_days"]
aggregate_cols = [
    "prod_profit_mean", "prod_profit_median", "prod_profit_std", "prod_profit_count",
    "state_profit_mean", "state_profit_median", "state_profit_std", "state_profit_count",
    "cat_sub_profit_mean", "cat_sub_profit_count",
    "cust_profit_mean", "cust_profit_count",
    "daily_profit_mean", "daily_profit_sum", "daily_profit_count"
]

In [195]:
encoded_cols = [c + "_enc" for c in cat_features]

features = numeric_cols + aggregate_cols + encoded_cols
features = [f for f in features if f in train.columns]

In [196]:
X = train[features]
y = np.log1p(train[target].fillna(0))

X_test = test[features]

/Users/daniil/.pyenv/versions/3.11.11/envs/ml-env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [197]:
params = {
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "seed": RANDOM_STATE,
    "lambda_l1": 0.1,
    "lambda_l2": 0.1
}

In [198]:
tscv = TimeSeriesSplit(n_splits=NFOLDS)
oof = np.zeros(len(X))
preds = np.zeros(len(X_test))
models = []

In [199]:
for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    train_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    model = lgb.train(
        params,
        train_data,
        num_boost_round=5000,
        valid_sets=[train_data, val_data],
        callbacks=[early_stopping(stopping_rounds=100),
                   log_evaluation(period=100)]
    )

    oof[val_idx] = np.expm1(model.predict(X_val, num_iteration=model.best_iteration))
    preds += np.expm1(model.predict(X_test, num_iteration=model.best_iteration)) / NFOLDS

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.365088	valid_1's rmse: 0.656726
[200]	training's rmse: 0.246096	valid_1's rmse: 0.641789
[300]	training's rmse: 0.176295	valid_1's rmse: 0.640028
[400]	training's rmse: 0.13242	valid_1's rmse: 0.641015
Early stopping, best iteration is:
[366]	training's rmse: 0.145328	valid_1's rmse: 0.636013
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.315493	valid_1's rmse: 0.575776
[200]	training's rmse: 0.206973	valid_1's rmse: 0.553903
[300]	training's rmse: 0.147613	valid_1's rmse: 0.55026
[400]	training's rmse: 0.109658	valid_1's rmse: 0.547815
[500]	training's rmse: 0.0859914	valid_1's rmse: 0.549679
Early stopping, best iteration is:
[400]	training's rmse: 0.109658	valid_1's rmse: 0.547815
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.300957	valid_1's rmse: 0.586886
[200]	training's rmse: 0.190181	valid_1's rmse: 0.578768
[300

In [201]:
mse = mean_squared_error(train[target], oof)
print(f"OOF MSE: {mse:.6f}")

OOF MSE: 26574.031866


In [200]:
rmse = np.sqrt(mean_squared_error(train[target], oof))
print(f"OOF RMSE: {rmse:.6f}")

OOF RMSE: 163.015434
